In [ ]:
# Kiểm tra logs trong database
# Sử dụng Django shell: python manage.py shell
# Sau đó chạy:

# from django.db import connection
# with connection.cursor() as cursor:
#     cursor.execute("SELECT COUNT(*) FROM logs")
#     count = cursor.fetchone()[0]
#     print(f"Tổng số logs: {count}")
#     
#     cursor.execute("SELECT level, channel, message, context, created_at FROM logs ORDER BY created_at DESC LIMIT 10")
#     rows = cursor.fetchall()
#     for row in rows:
#         print(f"[{row[4]}] {row[0].upper()} - {row[1]}: {row[2]}")
#         if row[3]:
#             print(f"  Context: {row[3]}")

print("Chạy code trên trong Django shell để kiểm tra logs")

In [1]:
from vnstock import Company
from vnstock import Listing
from vnstock import Finance
company = Company(symbol = "TPB", source="TCBS")
company2 = Company(symbol = "TEG", source="VCI")
listing = Listing()
fin = finance = Finance(symbol="VTO", source="VCI")



In [3]:
import re

df = listing.symbols_by_exchange()
df = df[df['exchange'] == 'HSX']

# lọc symbol chỉ toàn chữ (viết hoa hoặc viết thường)
df = df[df['symbol'].str.match(r'^[A-Z]+$')]

# Chỉ lấy danh sách symbol_id (mã chứng khoán), không phải bot
symbol_ids = df['symbol'].tolist()
print(f"Tổng số symbol_ids HSX (chỉ chữ cái): {len(symbol_ids)}")
print(f"Danh sách symbol_ids: {symbol_ids}")

     symbol exchange   type             organ_short_name  \
1       YEG      HSX  STOCK               Tập đoàn Yeah1   
2       YBM      HSX  STOCK        Khoáng sản CN Yên Bái   
33      VTP      HSX  STOCK            Bưu chính Viettel   
34      VTO      HSX  STOCK                       VITACO   
45      VTB      HSX  STOCK         Viettronics Tân Bình   
...     ...      ...    ...                          ...   
2991    ABS      HSX  STOCK    DV Nông nghiệp Bình Thuận   
2992    ABR      HSX  STOCK        Đầu tư Nhãn Hiệu Việt   
2997    AAT      HSX  STOCK  Tập đoàn Tiên Sơn Thanh Hóa   
2999    AAM      HSX  STOCK              Thủy sản Mekong   
3001    AAA      HSX  STOCK          An Phát Bioplastics   

                                          organ_name product_grp_id  
1                     Công ty Cổ phần Tập đoàn Yeah1            STO  
2     Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái            STO  
33            Tổng Công ty Cổ phần Bưu chính Viettel            STO  

In [ ]:
# Chỉ lấy danh sách symbol_id (mã chứng khoán) theo điều kiện đã lọc
symbol_ids = df['symbol'].tolist()
print(f"Tổng số symbol_ids HSX (chỉ chữ cái): {len(symbol_ids)}")
print(f"Danh sách symbol_ids: {symbol_ids}")

# Hoặc nếu muốn lấy 10 symbol_ids đầu tiên để test
first_10_symbol_ids = symbol_ids[:10]
print(f"\n10 symbol_ids đầu tiên: {first_10_symbol_ids}")

# Lưu vào biến để sử dụng sau này cho hệ thống bot trading
filtered_symbol_ids = symbol_ids

# Note: Đây là symbol_id (mã chứng khoán), không phải bot_id
# Trong database schema, symbol_id sẽ ref đến bảng symbol.id để quản lý trading bot cho từng mã CK

In [9]:
# Kiểm tra cấu trúc bảng logs hiện tại
from django.db import connection
cursor = connection.cursor()

# Kiểm tra các cột trong bảng logs
cursor.execute("""
    SELECT column_name, data_type, is_nullable 
    FROM information_schema.columns 
    WHERE table_name = 'logs'
    ORDER BY ordinal_position
""")
columns = cursor.fetchall()
print("Cấu trúc bảng logs hiện tại:")
for col in columns:
    print(f"  {col[0]} - {col[1]} (nullable: {col[2]})")

print("\n" + "="*50)

# Drop và tạo lại bảng với cấu trúc đúng
cursor.execute("DROP TABLE IF EXISTS logs CASCADE")
print("Đã xóa bảng logs cũ")

cursor.execute("""
    CREATE TABLE logs (
        id BIGSERIAL PRIMARY KEY,
        level VARCHAR(20) NOT NULL,
        channel VARCHAR(50) NOT NULL,
        message TEXT NOT NULL,
        context JSONB DEFAULT '{}',
        extra JSONB DEFAULT '{}',
        environment VARCHAR(50),
        created_at TIMESTAMP WITH TIME ZONE DEFAULT NOW()
    );
    
    CREATE INDEX idx_logs_created_at ON logs(created_at);
    CREATE INDEX idx_logs_level ON logs(level);
    CREATE INDEX idx_logs_channel ON logs(channel);
    CREATE INDEX idx_logs_context_gin ON logs USING GIN(context);
""")
print("Đã tạo lại bảng logs với cấu trúc đúng")

# Test insert
cursor.execute("""
    INSERT INTO logs (level, channel, message, context, environment)
    VALUES (%s, %s, %s, %s, %s)
""", ['info', 'test', 'Test after recreate', '{"test": true}', 'local'])

print("Test insert thành công!")

# Kiểm tra
cursor.execute("SELECT COUNT(*) FROM logs")
count = cursor.fetchone()[0]
print(f"Số logs sau khi tạo lại: {count}")

SynchronousOnlyOperation: You cannot call this from an async context - use a thread or sync_to_async.